In [8]:
pip install --upgrade pyecharts


   ---------------------------------------- 0.0/151.0 kB ? eta -:--:--
   -- ------------------------------------- 10.2/151.0 kB ? eta -:--:--
   ------- ------------------------------- 30.7/151.0 kB 325.1 kB/s eta 0:00:01
   --------------- ----------------------- 61.4/151.0 kB 469.7 kB/s eta 0:00:01
   ------------------------------ ------- 122.9/151.0 kB 722.1 kB/s eta 0:00:01
   -------------------------------------- 151.0/151.0 kB 749.0 kB/s eta 0:00:00
  Attempting uninstall: pyecharts
    Found existing installation: pyecharts 2.0.6
    Uninstalling pyecharts-2.0.6:
      Successfully uninstalled pyecharts-2.0.6
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体
plt.rcParams['axes.unicode_minus'] = False

# 假设你的数据框名为df
def clean_city_data(df, province_city_map):
    # 将邮政编码转换为城市名称
    def get_city(code):
        try:
            code = int(str(code)[:6])  # 取前6位
            return province_city_map.get(code, "未知")
        except:
            return "未知"
    
    # 如果有postal_code列，使用它来获取城市名称
    if 'postal_code' in df.columns:
        df['city'] = df['postal_code'].apply(get_city)
    
    # 统计每个城市的数量
    city_counts = df['city'].value_counts()
    
    # 绘制柱状图
    plt.figure(figsize=(15, 8))
    city_counts[:20].plot(kind='bar')
    plt.title('各城市数据量分布(Top 20)')
    plt.xlabel('城市')
    plt.ylabel('数量')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # 打印统计结果
    print("\n城市数据统计:")
    print(city_counts)
    
    return city_counts

# 使用示例
# df = pd.read_csv('your_data.csv')  # 读取你的数据
# city_stats = clean_city_data(df, PROVINCE_CITY_MAP)

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# 读取Excel文件
file_path = '数据文件/三周汇总数据.xlsx'
df = pd.read_excel(file_path)

# 查找包含实际数据的起始行
start_row = None
for i, row in df.iterrows():
    if row[1] == '工作簿' and row[2] == '工作表':
        start_row = i + 1  # 数据从这一行的下一行开始
        break

if start_row is not None:
    # 从实际数据开始的行切片DataFrame
    data_df = df.iloc[start_row:]
    
    # 假设城市信息在 'Unnamed: 4' 列
    if 'Unnamed: 4' in data_df.columns:
        # 将邮政编码转换为城市名称
        data_df['city'] = data_df['Unnamed: 4'].astype(str).str[:2]  # 取前两位作为编码
        data_df['city'] = data_df['city'].map(city_map)
        
        # 清理数据，去除任何可能的NaN值
        data_df.dropna(subset=['city'], inplace=True)
        
        # 统计每个城市的数据量
        city_counts = data_df['city'].value_counts()
        
        # 绘制条形图
        city_counts.plot(kind='bar', figsize=(10, 8))
        plt.title('City Data Counts')
        plt.xlabel('City')
        plt.ylabel('Counts')
        plt.xticks(rotation=45)
        plt.tight_layout()  # 调整布局以适应标签
        plt.show()
    else:
        print("DataFrame中没有找到包含城市信息的列，请检查列名是否正确")
else:
    print("未找到包含实际数据的行")

C:\Users\XiLeo\AppData\Local\Temp\ipykernel_8808\779368980.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[1] == '工作簿' and row[2] == '工作表':
C:\Users\XiLeo\AppData\Local\Temp\ipykernel_8808\779368980.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['city'] = data_df['Unnamed: 4'].astype(str).str[:2]  # 取前两位作为编码


NameError: name 'city_map' is not defined

In [3]:
import pandas as pd
import numpy as np

# 定义城市编码映射
city_map = {
    '10': '北京', '11': '北京', '12': '天津', '13': '河北', '14': '山西', '15': '内蒙古',
    '21': '辽宁', '22': '吉林', '23': '黑龙江', 
    '31': '上海', '32': '江苏', '33': '浙江', '34': '安徽', '35': '福建', '36': '江西', '37': '山东',
    '41': '河南', '42': '湖北', '43': '湖南', '44': '广东', '45': '广西', '46': '海南',
    '50': '重庆', '51': '四川', '52': '贵州', '53': '云南', '54': '西藏',
    '61': '陕西', '62': '甘肃', '63': '青海', '64': '宁夏', '65': '新疆',
    '71': '台湾', '81': '香港', '82': '澳门'
}

def process_excel(file_path):
    try:
        # 读取所有工作表
        all_sheets = pd.read_excel(file_path, sheet_name=None)
        
        # 用于存储所有符合条件的数据
        all_data = []
        
        # 遍历所有工作表
        for sheet_name, sheet_data in all_sheets.items():
            # 检查工作表是否包含所需列
            if 'Unnamed: 4' in sheet_data.columns:
                # 创建数据副本以避免SettingWithCopyWarning
                df = sheet_data.copy()
                
                # 提取邮政编码前两位
                df['city_code'] = df['Unnamed: 4'].astype(str).str[:2]
                
                # 使用映射转换为城市名称
                df['city'] = df['city_code'].map(city_map)
                
                # 删除临时的city_code列
                df = df.drop('city_code', axis=1)
                
                # 清理数据，去除NaN值
                df = df.dropna(subset=['city'])
                
                all_data.append(df)
        
        # 合并所有数据
        if all_data:
            final_df = pd.concat(all_data, ignore_index=True)
            
            # 按城市分组并计算统计信息
            city_stats = final_df.groupby('city').agg({
                'Unnamed: 1': 'count',  # 假设这是需要计数的列
                'Unnamed: 2': 'sum'     # 假设这是需要求和的列
            }).reset_index()
            
            # 重命名列
            city_stats.columns = ['城市', '数量', '总和']
            
            return city_stats
            
        else:
            return pd.DataFrame()
            
    except Exception as e:
        print(f"处理文件时出错: {str(e)}")
        return pd.DataFrame()

def export_results(city_stats, output_path):
    try:
        # 导出到Excel
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            city_stats.to_excel(writer, sheet_name='城市统计', index=False)
        return True
    except Exception as e:
        print(f"导出结果时出错: {str(e)}")
        return False

# 使用示例
def main():
    input_file = "input.xlsx"  # 输入文件路径
    output_file = "output.xlsx"  # 输出文件路径
    
    # 处理数据
    results = process_excel(input_file)
    
    if not results.empty:
        # 导出结果
        if export_results(results, output_file):
            print("处理完成，结果已导出到", output_file)
        else:
            print("导出结果失败")
    else:
        print("未找到有效数据或处理过程出错")

if __name__ == "__main__":
    main()

处理文件时出错: [Errno 2] No such file or directory: 'input.xlsx'
未找到有效数据或处理过程出错


In [4]:
import pandas as pd
import numpy as np
import os

# 定义城市编码映射
city_map = {
    '10': '北京', '11': '北京', '12': '天津', '13': '河北', '14': '山西', '15': '内蒙古',
    '21': '辽宁', '22': '吉林', '23': '黑龙江', 
    '31': '上海', '32': '江苏', '33': '浙江', '34': '安徽', '35': '福建', '36': '江西', '37': '山东',
    '41': '河南', '42': '湖北', '43': '湖南', '44': '广东', '45': '广西', '46': '海南',
    '50': '重庆', '51': '四川', '52': '贵州', '53': '云南', '54': '西藏',
    '61': '陕西', '62': '甘肃', '63': '青海', '64': '宁夏', '65': '新疆',
    '71': '台湾', '81': '香港', '82': '澳门'
}

def process_excel(file_path):
    try:
        if not os.path.exists(file_path):
            print(f"文件不存在: {file_path}")
            return pd.DataFrame()

        print(f"正在处理文件: {file_path}")
        
        # 读取所有工作表
        all_sheets = pd.read_excel(file_path, sheet_name=None)
        print(f"成功读取到 {len(all_sheets)} 个工作表")
        
        # 用于存储所有符合条件的数据
        all_data = []
        
        # 遍历所有工作表
        for sheet_name, sheet_data in all_sheets.items():
            print(f"正在处理工作表: {sheet_name}")
            
            # 打印列名，用于调试
            print(f"工作表列名: {sheet_data.columns.tolist()}")
            
            # 检查工作表是否包含所需列
            postal_code_column = None
            for col in sheet_data.columns:
                if isinstance(col, str) and '邮编' in col:
                    postal_code_column = col
                    break
            
            if postal_code_column:
                # 创建数据副本以避免SettingWithCopyWarning
                df = sheet_data.copy()
                
                # 提取邮政编码前两位
                df['city_code'] = df[postal_code_column].astype(str).str[:2]
                
                # 使用映射转换为城市名称
                df['city'] = df['city_code'].map(city_map)
                
                # 删除临时的city_code列
                df = df.drop('city_code', axis=1)
                
                # 清理数据，去除NaN值
                df = df.dropna(subset=['city'])
                
                print(f"工作表 {sheet_name} 处理完成，找到 {len(df)} 条有效数据")
                all_data.append(df)
            else:
                print(f"工作表 {sheet_name} 不包含邮编列")
        
        # 合并所有数据
        if all_data:
            final_df = pd.concat(all_data, ignore_index=True)
            print(f"总共处理了 {len(final_df)} 条数据")
            
            # 按城市分组并计算统计信息
            # 获取数值类型的列用于统计
            numeric_columns = final_df.select_dtypes(include=[np.number]).columns
            if len(numeric_columns) > 0:
                first_numeric_col = numeric_columns[0]
                city_stats = final_df.groupby('city').agg({
                    first_numeric_col: ['count', 'sum']
                }).reset_index()
                
                # 重命名列
                city_stats.columns = ['城市', '数量', '总和']
            else:
                # 如果没有数值列，只计算数量
                city_stats = final_df.groupby('city').size().reset_index(name='数量')
                city_stats['总和'] = 0
            
            print("数据处理完成")
            return city_stats
            
        else:
            print("未找到任何有效数据")
            return pd.DataFrame()
            
    except Exception as e:
        print(f"处理文件时出错: {str(e)}")
        print("错误详细信息:")
        import traceback
        traceback.print_exc()
        return pd.DataFrame()

def export_results(city_stats, output_path):
    try:
        # 确保输出目录存在
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # 导出到Excel
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            city_stats.to_excel(writer, sheet_name='城市统计', index=False)
        print(f"结果已成功导出到: {output_path}")
        return True
    except Exception as e:
        print(f"导出结果时出错: {str(e)}")
        return False

def main():
    print("开始执行程序...")
    
    # 使用指定的文件路径
    input_file = os.path.join("数据文件", "三周汇总数据.xlsx")
    output_file = os.path.join("数据文件", "城市统计结果.xlsx")
    
    # 处理数据
    results = process_excel(input_file)
    
    if not results.empty:
        # 导出结果
        if export_results(results, output_file):
            print("处理完成，结果已导出")
            # 显示处理结果预览
            print("\n处理结果预览:")
            print(results)
        else:
            print("导出结果失败")
    else:
        print("未找到有效数据或处理过程出错")

if __name__ == "__main__":
    main()

开始执行程序...
正在处理文件: 数据文件\三周汇总数据.xlsx
成功读取到 2 个工作表
正在处理工作表: 报告
工作表列名: ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']
工作表 报告 不包含邮编列
正在处理工作表: 总表
工作表列名: ['Unnamed: 0', 'user_id', 'age', 'gender', 'city', 'Unnamed: 5']
工作表 总表 不包含邮编列
未找到任何有效数据
未找到有效数据或处理过程出错


In [5]:
import pandas as pd
import numpy as np
import os

# 定义城市编码映射
city_map = {
    '10': '北京', '11': '北京', '12': '天津', '13': '河北', '14': '山西', '15': '内蒙古',
    '21': '辽宁', '22': '吉林', '23': '黑龙江', 
    '31': '上海', '32': '江苏', '33': '浙江', '34': '安徽', '35': '福建', '36': '江西', '37': '山东',
    '41': '河南', '42': '湖北', '43': '湖南', '44': '广东', '45': '广西', '46': '海南',
    '50': '重庆', '51': '四川', '52': '贵州', '53': '云南', '54': '西藏',
    '61': '陕西', '62': '甘肃', '63': '青海', '64': '宁夏', '65': '新疆',
    '71': '台湾', '81': '香港', '82': '澳门'
}

def process_excel(file_path):
    try:
        if not os.path.exists(file_path):
            print(f"文件不存在: {file_path}")
            return pd.DataFrame()

        print(f"正在处理文件: {file_path}")
        
        # 读取所有工作表
        all_sheets = pd.read_excel(file_path, sheet_name=None)
        print(f"成功读取到 {len(all_sheets)} 个工作表")
        
        # 用于存储所有符合条件的数据
        all_data = []
        
        # 遍历所有工作表
        for sheet_name, sheet_data in all_sheets.items():
            print(f"正在处理工作表: {sheet_name}")
            
            # 打印列名和前几行数据，用于调试
            print(f"工作表列名: {sheet_data.columns.tolist()}")
            print("\n前几行数据预览:")
            print(sheet_data.head())
            
            if 'Unnamed: 4' in sheet_data.columns:  # 处理报告工作表
                df = sheet_data.copy()
                postal_codes = df['Unnamed: 4'].astype(str)
                valid_codes = postal_codes.str.match(r'^\d{6}$')  # 检查是否为6位数字
                df = df[valid_codes]
                if not df.empty:
                    df['city_code'] = df['Unnamed: 4'].astype(str).str[:2]
                    df['city'] = df['city_code'].map(city_map)
                    df = df.drop('city_code', axis=1)
                    all_data.append(df)
                    print(f"报告表处理完成，找到 {len(df)} 条有效数据")
            
            elif 'city' in sheet_data.columns:  # 处理总表工作表
                df = sheet_data.copy()
                all_data.append(df)
                print(f"总表处理完成，找到 {len(df)} 条数据")
        
        # 合并所有数据
        if all_data:
            final_df = pd.concat(all_data, ignore_index=True)
            print(f"总共处理了 {len(final_df)} 条数据")
            
            # 按城市分组并计算统计信息
            if 'age' in final_df.columns:  # 如果有age列，计算年龄统计
                city_stats = final_df.groupby('city').agg({
                    'age': ['count', 'mean']
                }).reset_index()
                city_stats.columns = ['城市', '数量', '平均年龄']
            else:  # 否则只计算数量
                city_stats = final_df.groupby('city').size().reset_index(name='数量')
            
            print("数据处理完成")
            return city_stats
            
        else:
            print("未找到任何有效数据")
            return pd.DataFrame()
            
    except Exception as e:
        print(f"处理文件时出错: {str(e)}")
        print("错误详细信息:")
        import traceback
        traceback.print_exc()
        return pd.DataFrame()

def export_results(city_stats, output_path):
    try:
        # 确保输出目录存在
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # 导出到Excel
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            city_stats.to_excel(writer, sheet_name='城市统计', index=False)
            
            # 添加数据分析
            if 'average_age' in city_stats.columns:
                analysis = pd.DataFrame({
                    '统计指标': ['总人数', '城市数量', '平均年龄'],
                    '值': [
                        city_stats['数量'].sum(),
                        len(city_stats),
                        city_stats['平均年龄'].mean()
                    ]
                })
                analysis.to_excel(writer, sheet_name='数据分析', index=False)
            
        print(f"结果已成功导出到: {output_path}")
        return True
    except Exception as e:
        print(f"导出结果时出错: {str(e)}")
        return False

def main():
    print("开始执行程序...")
    
    # 使用指定的文件路径
    input_file = os.path.join("数据文件", "三周汇总数据.xlsx")
    output_file = os.path.join("数据文件", "城市统计结果.xlsx")
    
    # 处理数据
    results = process_excel(input_file)
    
    if not results.empty:
        # 导出结果
        if export_results(results, output_file):
            print("处理完成，结果已导出")
            # 显示处理结果预览
            print("\n处理结果预览:")
            print(results)
        else:
            print("导出结果失败")
    else:
        print("未找到有效数据或处理过程出错")

if __name__ == "__main__":
    main()

开始执行程序...
正在处理文件: 数据文件\三周汇总数据.xlsx
成功读取到 2 个工作表
正在处理工作表: 报告
工作表列名: ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']

前几行数据预览:
   Unnamed: 0                      Unnamed: 1      Unnamed: 2 Unnamed: 3  \
0         NaN  报告：成功合并 102 个工作表，共 300838 行数据。             NaN        NaN   
1         NaN                             NaN             NaN        NaN   
2         NaN                             工作簿             工作表       合并状态   
3         NaN              comments_data1.csv  comments_data1         成功   
4         NaN              comments_data2.csv  comments_data2         成功   

         Unnamed: 4  
0               NaN  
1               NaN  
2            合并后的位置  
3      总表!B1:F10200  
4  总表!B10201:F15300  
正在处理工作表: 总表
工作表列名: ['Unnamed: 0', 'user_id', 'age', 'gender', 'city', 'Unnamed: 5']

前几行数据预览:
                       Unnamed: 0     user_id age gender    city Unnamed: 5
0  [comments_data1]comments_data1   544930017   0      1     100        NaN
1  [comments_data

Traceback (most recent call last):
  File "C:\Users\XiLeo\anaconda3\Lib\site-packages\pandas\core\groupby\groupby.py", line 1942, in _agg_py_fallback
    res_values = self._grouper.agg_series(ser, alt, preserve_dtype=True)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\XiLeo\anaconda3\Lib\site-packages\pandas\core\groupby\ops.py", line 864, in agg_series
    result = self._aggregate_series_pure_python(obj, func)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\XiLeo\anaconda3\Lib\site-packages\pandas\core\groupby\ops.py", line 885, in _aggregate_series_pure_python
    res = func(group)
          ^^^^^^^^^^^
  File "C:\Users\XiLeo\anaconda3\Lib\site-packages\pandas\core\groupby\groupby.py", line 2454, in <lambda>
    alt=lambda x: Series(x, copy=False).mean(numeric_only=numeric_only),
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\XiLeo\anaconda3\Lib\site-packages\pandas\core\

In [6]:
import pandas as pd
import numpy as np
import os

# 定义城市编码映射
city_map = {
    '10': '北京', '11': '北京', '12': '天津', '13': '河北', '14': '山西', '15': '内蒙古',
    '21': '辽宁', '22': '吉林', '23': '黑龙江', 
    '31': '上海', '32': '江苏', '33': '浙江', '34': '安徽', '35': '福建', '36': '江西', '37': '山东',
    '41': '河南', '42': '湖北', '43': '湖南', '44': '广东', '45': '广西', '46': '海南',
    '50': '重庆', '51': '四川', '52': '贵州', '53': '云南', '54': '西藏',
    '61': '陕西', '62': '甘肃', '63': '青海', '64': '宁夏', '65': '新疆',
    '71': '台湾', '81': '香港', '82': '澳门'
}

def clean_city_code(code):
    # 转换为字符串并取前两位
    if pd.isna(code):
        return None
    code_str = str(code)
    if len(code_str) >= 2:
        return code_str[:2]
    return None

def clean_age(age):
    try:
        age_val = float(age)
        if 0 <= age_val <= 120:  # 合理的年龄范围
            return age_val
        return None
    except (ValueError, TypeError):
        return None

def process_excel(file_path):
    try:
        if not os.path.exists(file_path):
            print(f"文件不存在: {file_path}")
            return pd.DataFrame()

        print(f"正在处理文件: {file_path}")
        
        # 读取所有工作表
        all_sheets = pd.read_excel(file_path, sheet_name=None)
        print(f"成功读取到 {len(all_sheets)} 个工作表")
        
        # 用于存储所有符合条件的数据
        all_data = []
        
        # 遍历所有工作表
        for sheet_name, sheet_data in all_sheets.items():
            print(f"正在处理工作表: {sheet_name}")
            print(f"工作表列名: {sheet_data.columns.tolist()}")
            print("\n前几行数据预览:")
            print(sheet_data.head())
            
            if 'city' in sheet_data.columns:  # 处理总表工作表
                df = sheet_data.copy()
                
                # 清理和转换城市代码
                df['city_code'] = df['city'].apply(clean_city_code)
                df['city_name'] = df['city_code'].map(city_map)
                
                # 清理年龄数据
                df['age_clean'] = df['age'].apply(clean_age)
                
                # 只保留有效的数据行
                df = df.dropna(subset=['city_name'])
                
                if not df.empty:
                    all_data.append(df)
                    print(f"总表处理完成，找到 {len(df)} 条数据")
        
        # 合并所有数据
        if all_data:
            final_df = pd.concat(all_data, ignore_index=True)
            print(f"总共处理了 {len(final_df)} 条数据")
            
            # 按城市分组统计
            city_stats = (final_df.groupby('city_name')
                         .agg({
                             'age_clean': ['count', lambda x: x.mean(skipna=True)],
                             'gender': 'count'
                         })
                         .reset_index())
            
            # 重命名列
            city_stats.columns = ['城市', '数量', '平均年龄', '性别计数']
            
            # 处理可能的NaN值
            city_stats['平均年龄'] = city_stats['平均年龄'].round(2)
            city_stats = city_stats.fillna({'平均年龄': 0})
            
            print("数据处理完成")
            return city_stats
            
        else:
            print("未找到任何有效数据")
            return pd.DataFrame()
            
    except Exception as e:
        print(f"处理文件时出错: {str(e)}")
        print("错误详细信息:")
        import traceback
        traceback.print_exc()
        return pd.DataFrame()

def export_results(city_stats, output_path):
    try:
        # 确保输出目录存在
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # 导出到Excel
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            city_stats.to_excel(writer, sheet_name='城市统计', index=False)
            
            # 添加数据分析
            analysis = pd.DataFrame({
                '统计指标': ['总人数', '城市数量', '总体平均年龄'],
                '值': [
                    city_stats['数量'].sum(),
                    len(city_stats),
                    (city_stats['平均年龄'] * city_stats['数量']).sum() / city_stats['数量'].sum()
                ]
            })
            analysis.to_excel(writer, sheet_name='数据分析', index=False)
            
        print(f"结果已成功导出到: {output_path}")
        return True
    except Exception as e:
        print(f"导出结果时出错: {str(e)}")
        return False

def main():
    print("开始执行程序...")
    
    # 使用指定的文件路径
    input_file = os.path.join("数据文件", "三周汇总数据.xlsx")
    output_file = os.path.join("数据文件", "城市统计结果.xlsx")
    
    # 处理数据
    results = process_excel(input_file)
    
    if not results.empty:
        # 导出结果
        if export_results(results, output_file):
            print("处理完成，结果已导出")
            # 显示处理结果预览
            print("\n处理结果预览:")
            print(results)
        else:
            print("导出结果失败")
    else:
        print("未找到有效数据或处理过程出错")

if __name__ == "__main__":
    main()

开始执行程序...
正在处理文件: 数据文件\三周汇总数据.xlsx
成功读取到 2 个工作表
正在处理工作表: 报告
工作表列名: ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']

前几行数据预览:
   Unnamed: 0                      Unnamed: 1      Unnamed: 2 Unnamed: 3  \
0         NaN  报告：成功合并 102 个工作表，共 300838 行数据。             NaN        NaN   
1         NaN                             NaN             NaN        NaN   
2         NaN                             工作簿             工作表       合并状态   
3         NaN              comments_data1.csv  comments_data1         成功   
4         NaN              comments_data2.csv  comments_data2         成功   

         Unnamed: 4  
0               NaN  
1               NaN  
2            合并后的位置  
3      总表!B1:F10200  
4  总表!B10201:F15300  
正在处理工作表: 总表
工作表列名: ['Unnamed: 0', 'user_id', 'age', 'gender', 'city', 'Unnamed: 5']

前几行数据预览:
                       Unnamed: 0     user_id age gender    city Unnamed: 5
0  [comments_data1]comments_data1   544930017   0      1     100        NaN
1  [comments_data

In [7]:
pip install matplotlib pandas geopandas plotly

   ---------------------------------------- 0.0/323.6 kB ? eta -:--:--
   ----- ---------------------------------- 41.0/323.6 kB ? eta -:--:--
   ---------- ----------------------------- 81.9/323.6 kB 1.1 MB/s eta 0:00:01
   -------------------- ------------------- 163.8/323.6 kB 1.4 MB/s eta 0:00:01
   --------------------------- ------------ 225.3/323.6 kB 1.4 MB/s eta 0:00:01
   ----------------------------------- ---- 286.7/323.6 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 323.6/323.6 kB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/16.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/16.2 MB 3.4 MB/s eta 0:00:05
   ---------------------------------------- 0.1/16.2 MB 2.6 MB/s eta 0:00:07
   ---------------------------------------- 0.2/16.2 MB 1.6 MB/s eta 0:00:11
    --------------------------------------- 0.2/16.2 MB 1.4 MB/s eta 0:00:12
    --------------------------------------- 0.3/16.2 MB 1.6 MB/s eta 0:00:10
   

In [13]:
pip install pandas pyecharts

Note: you may need to restart the kernel to use updated packages.


In [16]:
from pyecharts.charts import Map

In [20]:
pip install -U pyecharts

Note: you may need to restart the kernel to use updated packages.


In [29]:
import pandas as pd
from pyecharts.charts import Map
from pyecharts import options as opts

# 映射函数，将你的城市名称映射到pyecharts地图中的全称
def map_city_name(city_name):
    city_mapping = {
        "上海": "上海市",
        "云南": "云南省",
        "内蒙古": "内蒙古自治区",
        "北京": "北京市",
        "台湾": "台湾省",
        "吉林": "吉林省",
        "四川": "四川省",
        "天津": "天津市",
        "宁夏": "宁夏回族自治区",
        "安徽": "安徽省",
        "山东": "山东省",
        "山西": "山西省",
        "广东": "广东省",
        "广西": "广西壮族自治区",
        "新疆": "新疆维吾尔自治区",
        "江苏": "江苏省",
        "江西": "江西省",
        "河北": "河北省",
        "河南": "河南省",
        "浙江": "浙江省",
        "海南": "海南省",
        "湖北": "湖北省",
        "湖南": "湖南省",
        "澳门": "澳门特别行政区",
        "甘肃": "甘肃省",
        "福建": "福建省",
        "西藏": "西藏自治区",
        "贵州": "贵州省",
        "辽宁": "辽宁省",
        "重庆": "重庆市",
        "陕西": "陕西省",
        "青海": "青海省",
        "香港": "香港特别行政区",
        "黑龙江": "黑龙江省"
    }
    return city_mapping.get(city_name, city_name)

# 读取Excel文件
df = pd.read_excel("数据文件\\城市统计结果.xlsx")

# 应用映射函数
df['城市'] = df['城市'].apply(map_city_name)

# 准备地图数据
attr = df['城市'].tolist()
value = df['数量'].tolist()

# 绘制地图
map = Map()
map.add("城市人数", [list(z) for z in zip(attr, value)], "china", is_map_symbol_show=True)
map.set_global_opts(
    title_opts=opts.TitleOpts(title="城市人数分布图"),
    visualmap_opts=opts.VisualMapOpts(is_show=True, is_piecewise=True, min_=min(value), max_=max(value))
)
map.render('城市人数分布图.html')

'C:\\Users\\XiLeo\\城市人数分布图.html'